## Import

In [1]:
import torch
from Models import MgNet, MSACNN, MgFF, Linear
from torchinfo import summary
print(torch.__version__)

category = 25 # for Damage dataset and the category of Multiple dataset is 16

2.5.1


In [2]:
class Get_Model(torch.nn.Module):
    def __init__(self, categories, model, dims = 64 ):
        super().__init__()

        if model == 'MgNet':
            self.backbone =  MgNet( dims )
        if model == 'MSACNN':
            self.backbone =  MSACNN( dims )
            
        self.predictor = Linear( dims, categories, zeros_init = True )
        
    def forward(self, x):
        
        z = self.backbone(x)
        prediction = self.predictor( z )
        
        return prediction

## MgNet

In [3]:
input_size = (1, 1, 2048 )
net = Get_Model( category, 'MgNet' )
model_info = summary(net, input_size=input_size, verbose=0, col_names=["input_size", "output_size", "num_params", "mult_adds"])
print(model_info)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Mult-Adds
Get_Model                                               [1, 1, 2048]              [1, 25]                   --                        --
├─MgNet: 1-1                                            [1, 1, 2048]              [1, 64]                   2                         --
│    └─Sequential: 2-1                                  [1, 1, 2048]              [1, 128, 8]               --                        --
│    │    └─MgFF: 3-1                                   [1, 1, 2048]              [1, 4, 512]               218                       131,112
│    │    └─MgFF: 3-2                                   [1, 4, 512]               [1, 16, 128]              2,582                     475,296
│    │    └─MgFF: 3-3                                   [1, 16, 128]              [1, 64, 32]               38,678                    1,852,032
│    │    └─MgFF:

In [4]:
# MgFF: 3-2     
input_size = (1, 4, 512 )
net = MgFF( 4, 16 )
model_info = summary(net, input_size=input_size, verbose=0, col_names=["input_size", "output_size", "num_params", "mult_adds"])
print(model_info)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
MgFF                                     [1, 4, 512]               [1, 16, 128]              --                        --
├─Sequential: 1-1                        [1, 4, 512]               [1, 16, 128]              --                        --
│    └─Conv1D: 2-1                       [1, 4, 512]               [1, 16, 512]              --                        --
│    │    └─Conv1d: 3-1                  [1, 4, 512]               [1, 16, 512]              80                        40,960
│    │    └─BatchNorm1d: 3-2             [1, 16, 512]              [1, 16, 512]              32                        32
│    │    └─ACON: 3-3                    [1, 16, 512]              [1, 16, 512]              2                         --
│    └─MaxPool1d: 2-2                    [1, 16, 512]              [1, 16, 128]              --                        --
├─ModuleList:

## MSACNN

In [5]:
input_size = (1, 1, 2048 )
net = Get_Model( category, 'MSACNN' )
model_info = summary(net, input_size=input_size, verbose=0, col_names=["input_size", "output_size", "num_params", "mult_adds"])
print(model_info)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
Get_Model                                [1, 1, 2048]              [1, 25]                   --                        --
├─MSACNN: 1-1                            [1, 1, 2048]              [1, 64]                   --                        --
│    └─MultiScaleConv: 2-1               [1, 1, 32, 64]            [1, 96, 32, 64]           --                        --
│    │    └─ModuleList: 3-1              --                        --                        2,752                     --
│    └─ChannelAttention: 2-2             [1, 96, 32, 64]           [1, 96, 1, 1]             --                        --
│    │    └─AdaptiveAvgPool2d: 3-2       [1, 96, 32, 64]           [1, 96, 1, 1]             --                        --
│    │    └─Sequential: 3-3              [1, 96]                   [1, 96]                   1,152                     1,152
│    │    └─Ad